### **`check_envs_registration.ipynb`**


This notebook verifies the successful registration of all custom `LLECBuildingGym` environments and performs basic environment interaction tests.  
It includes:
- Import validation and environment listing  
- Basic reset and step checks for selected environments  
- Reward consistency validation  
- Gymnasium environment compliance check (`check_env`)

In [1]:
import sys
import numpy as np
import gymnasium as gym

print("Python executable:", sys.executable)
# Display all available Jupyter kernels
!jupyter kernelspec list | grep llec_env

# Attempt to the LLECBuildingGym module, which registers custom environments via `register()`
try:
    import llec_building_gym  # Triggers registration on llec_building_gym/__init__.py and registers environments

    print("llec_building_gym path:", llec_building_gym.__file__)
    print("Module 'llec_building_gym' successfully loaded.")
except ImportError:
    print("Module 'llec_building_gym' could not be loaded.")
    print("Please install it using: pip install -e .")
else:
    print("\nRegistered LLEC environments:")
    # List all environments containing 'LLEC' in their ID
    custom_envs = [
        env_id for env_id in sorted(gym.envs.registry.keys()) if "LLEC" in env_id
    ]
    if not custom_envs:
        print("No LLEC environments found.")
    else:
        for env_id in custom_envs:
            print(f"  {env_id}")
        print(f"\nTotal found: {len(custom_envs)} LLEC environments")

Python executable: /home/iai/ii6824/llec_env/bin/python
  python3           /hkfs/home/haicore/iai/ii6824/llec_env/share/jupyter/kernels/python3
llec_building_gym path: /hkfs/home/haicore/iai/ii6824/LLECBuildingGym/llec_building_gym/__init__.py
Module 'llec_building_gym' successfully loaded.

Registered LLEC environments:
  LLEC-HeatPumpHouse-1R1C-Combined-v0
  LLEC-HeatPumpHouse-1R1C-Temperature-v0

Total found: 2 LLEC environments


In [2]:
print(llec_building_gym.__file__)

# Choose one of the available environments below:
env = gym.make("LLEC-HeatPumpHouse-1R1C-Temperature-v0")
# env = gym.make("LLEC-HeatPumpHouse-1R1C-Combined-v0")

# Print observation and action space information
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)

# Reset the environment and display the initial observation
obs, info = env.reset()
print("Initial observation:", obs)

2025-11-05 22:11:49,983 - INFO - [EnvID:22687759392928] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=6314
2025-11-05 22:11:49,992 - INFO - [EnvID:22687759392928] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=1895


/hkfs/home/haicore/iai/ii6824/LLECBuildingGym/llec_building_gym/__init__.py
Observation Space: Box(-inf, inf, (1,), float32)
Action Space: Box(-1.0, 1.0, (1,), float32)
Initial observation: [8.84775]


In [3]:
# Print all registered environments
print("=== Registered Gym Environments ===")
env_list = sorted(gym.envs.registry.keys())

# Check if the desired environment is registered
env_name = "LLEC-HeatPumpHouse-1R1C-Temperature-v0"
if env_name in env_list:
    print(f"\nEnvironment '{env_name}' is successfully registered.")
else:
    print(
        f"\nEnvironment '{env_name}' is not registered. Please check your setup or registration logic."
    )

=== Registered Gym Environments ===

Environment 'LLEC-HeatPumpHouse-1R1C-Temperature-v0' is successfully registered.


In [4]:
from llec_building_gym import BaseBuildingGym  # Adjust the path if necessary

# Instantiate the environment
env = BaseBuildingGym(energy_price_path="../data/price_data_2025.csv")
obs, _ = env.reset()
step_count = 0

# Simulate one full episode using random actions
while True:
    action = env.action_space.sample()  # Sample a random action
    obs, reward, terminated, truncated, info = env.step(action)
    step_count += 1
    if terminated or truncated:
        break

# Print episode summary
print(f"Episode finished after {step_count} steps.")
print(f"Final indoor temperature: {env.building.T_in:.2f}°C")
print(f"Final observation: {obs}")
print(f"Final observation length: {len(obs)}")
print(f"Final reward: {reward:.2f}")

2025-11-05 22:12:01,423 - INFO - [EnvID:22687758954256] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=6184
2025-11-05 22:12:01,463 - INFO - [EnvID:22687758954256] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=6184


Episode finished after 288 steps.
Final indoor temperature: 11.66°C
Final observation: [-10.363885]
Final observation length: 1
Final reward: 0.00


In [5]:
# List of all registered environment IDs to test
registered_envs = [
    "LLEC-HeatPumpHouse-1R1C-Combined-v0",
    "LLEC-HeatPumpHouse-1R1C-Temperature-v0",
]


def test_env(env_id, n_steps=3):
    print(f"\n=== Testing: {env_id} ===")
    try:
        env = gym.make(env_id, energy_price_path="../data/price_data_2025.csv")
        obs, info = env.reset(seed=42)
        print(f"Initial observation: {obs}")

        for step in range(n_steps):
            action = env.action_space.sample()
            obs, reward, terminated, truncated, info = env.step(action)

            print(f"\nStep {step + 1}")
            print(f"  Action: {action}")
            print(f"  Observation: {obs}")
            print(f"  Reward: {reward:.4f}")
            print(f"  Terminated: {terminated}, Truncated: {truncated}")
            print(f"  Info:")
            for k, v in info.items():
                if isinstance(v, float):
                    print(f"    {k}: {v:.4f}")
                else:
                    print(f"    {k}: {v}")

            # Optional: Reward consistency check
            if "reward_temperature" in info and "reward_energy" in info:
                reward_check = info["reward_temperature"] + info["reward_energy"]
                diff = abs(reward - reward_check)
                status = "ok" if diff <= 1e-3 else "(!)"
                print(f"    Reward consistency check: Δ = {diff:.6f} [{status}]")

            if terminated or truncated:
                print("  Episode ended early.")
                break

        env.close()

    except Exception as e:
        print(f"Error testing {env_id}: {e}")


if __name__ == "__main__":
    for env_id in registered_envs:
        test_env(env_id, n_steps=3)

2025-11-05 22:12:09,905 - INFO - [EnvID:22687241223520] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=6184
2025-11-05 22:12:09,941 - INFO - [EnvID:22687241223520] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=42
2025-11-05 22:12:10,007 - INFO - [EnvID:22687241223568] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=6184
2025-11-05 22:12:10,040 - INFO - [EnvID:22687241223568] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=42



=== Testing: LLEC-HeatPumpHouse-1R1C-Combined-v0 ===
Initial observation: [0.8960016]

Step 1
  Action: [-0.02914016]
  Observation: [8.086991]
  Reward: -0.0099
  Terminated: False, Truncated: False
  Info:
    temp_deviation: 7.5225
    action: [-0.02914016]
    T_out: 14.9378
    Q_HP_Max: 1500
    controlled_Q_HP: -43.7102
    reward: -0.0099
    reward_temperature: 0.0005
    reward_economic: -0.0104
    reward_temperature_norm: 0.0005
    reward_economic_norm: -0.0104
    history_temp_deviations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [6]:
from stable_baselines3.common.env_checker import check_env
from llec_building_gym import BaseBuildingGym  # Adjust the import path if necessary

# Create an instance of your custom environment
env = BaseBuildingGym(energy_price_path="../data/price_data_2025.csv")

# Run the environment check to validate compatibility with Stable Baselines3
check_env(env, warn=True, skip_render_check=True)

2025-11-05 22:13:11,663 - INFO - [EnvID:22687241222944] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=6184
2025-11-05 22:13:11,696 - INFO - [EnvID:22687241222944] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=0
2025-11-05 22:13:11,724 - INFO - [EnvID:22687241222944] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=6184
2025-11-05 22:13:11,753 - INFO - [EnvID:22687241222944] schedule=24-hour | Tmin=14.0°C  Tmax=28.0°C  phase=2.094  seed=6184


In [7]:
print(
    "Environment check completed successfully in `check_envs_registration.ipynb`. The environment is compatible with Stable Baselines3."
)

Environment check completed successfully in `check_envs_registration.ipynb`. The environment is compatible with Stable Baselines3.
